In [1]:
"""This is video reader with all operations inside fnctions, so they can be called on 
    each frame out of a set of frames, and each video out of a set of videos
    """

'This is video reader with all operations inside fnctions, so they can be called on \n    each frame out of a set of frames, and each video out of a set of videos\n    '

In [2]:
"""Imports"""
import matplotlib.pyplot as plt #for maknig plots inside the notebook
import skimage
import skvideo.io
from skvideo.io import vreader, ffprobe
from skimage import measure, morphology, feature
from skimage.filters import *
from skimage.morphology import *
from operator import attrgetter
import numpy as np
import math
from itertools import chain
#from functools import partial, update_wrapper
from scipy import ndimage 
from pims import pipeline, Video
from glob import glob
import seaborn as sns
from collections import OrderedDict 
#import antigravity

In [3]:
"""Functions"""
# apply total thresholding to each of a list of frames; filter slides with wrong number

def total_threshold_filter(frame, frame_no, transition_threshold, class_thresh,
                           broken_count, class_num, origin, last_class, last_whole, filtrate_len, 
                           sides, classes, transitions): 
    
   
    
   # ---setup image and detect blobs ----------------
    thresh_img = frame > threshold_isodata(frame)# binary image
        
    phighlight = morphology.binary_opening(thresh_img, square(14))
    img = np.copy(frame)
    img[phighlight==0] = 0
    pblobs =skimage.feature.blob_doh(img, min_sigma = 10, max_sigma = 16, threshold = 0.007, num_sigma= 15, overlap=0.3)

    ghighlight = morphology.binary_opening(
    morphology.white_tophat(gaussian(thresh_img, sigma=0.3), square(14)),
                               disk(4))

    img = np.copy(frame)
    img[ghighlight==0] = 0
    gblobs =skimage.feature.blob_doh(img, min_sigma = 6, max_sigma = 16, threshold = 0.007, num_sigma= 15, overlap=0.5)


    ngblobs = len(gblobs)
    npblobs = len(pblobs) 
    num_blobs = ngblobs + npblobs
            
    plas_on_glas_dists =[connect((pblob[1], pblob[0]),m_bond, gblobs) for pblob in pblobs]
    num_mconnections = reducer(plas_on_glas_dists)


    plas_on_plas_dists =[connect((pblobs[i][1], pblobs[i][0]),p_bond, pblobs[i+1:]) for i in range(npblobs-1)]
    num_pconnections = reducer(plas_on_plas_dists)



    num_connections = sum([num_pconnections,num_mconnections])
    
    #----------------------filter------------------------- can use actual filter
    test = (num_connections >= expected_connections and num_blobs>=expected_blobs)
    if test: # keep frames that have enough paricles, and are not transitions
        side =sideify(num_pconnections, num_mconnections )
        clas = side
        sides[side].append(frame_no)
        classes[clas].append(frame_no)
            #print(clas)
        
        if clas == 'ucf':
            #broken_count += 1
            #class_num = 0
            #print('ucf')
            pass
        else:
            if origin == 'ucf':
                origin = last_class
                #print("reset origin", origin)
            if clas == last_class:
                class_num += 1
                #print("class_num: %s" %class_num)
            else:
                class_num = 0
                #print("class_reset")
                #print("class_num: %s" %class_num)
            if class_num >= class_thresh: 
                if broken_count >= transition_threshold or clas != origin:
                    transitions[origin + "->" + clas].append((last_whole, frame_no))
                    #print(origin + "->" + clas)
                    origin = clas
                    #print("origin: %s" %origin)
                broken_count = 0
                #print("broken_count: %s" %broken_count)
                
            
                
            last_whole = frame_no
            filtrate_len += 1 
            last_class = clas
            
            
    else:
        #print("broken")
        class_num = 0
        #print("class_num: %s" %class_num)
        broken_count += 1
        #print("broken_count: %s" %broken_count)

    return [broken_count, class_num, origin, last_class, last_whole, filtrate_len]



        
# take largest isodata image region, and return aclassification
def sideify( num_pconnections, num_mconnections):    
    if num_mconnections == 2 and num_pconnections == 3:# and num_mconnections==4: #gtotallengths>48
        return "t"
    elif num_mconnections == 3 and num_pconnections == 2:# and num_mconnections == 4:
        return "c"
    else:
        return "ucf"

def reducer(connections):
    return sum(connections)    
def connect(cv, radius, bloblist): # center of blob2, radius of all blobs, list of other blobs
    num_connections = 0
    norm_radius = 15
    for blob2 in bloblist:
        cv2 = (blob2[1], blob2[0]) # center of blob2
        vector_d = vector_dist(cv, cv2)
        if vector_d <= 2*radius: # blob centers closer than diameter of one blob
            num_connections += 1
        else:
            pass
    return (num_connections)

def vector_dist(v1, v2): # euclidean distance between 2 points
    return math.sqrt(np.sum([(v1[i] - v2[i])**2 for i in range(len(v1))]))

def third_item(l1):
    return l1[2]   

In [ ]:
"""Pipeline for lists of frames"""
""""""
"""Pipeline for videos:"""
# initialize frame lists
filenames = glob("./tchakamau/W*plastic/4*avi")# +glob("./tchakamau/W*half/*_[1][0,1,2,3,4,7,8,9]*.avi")  #[1,2,3,4,6,7,8,9]["./tchakamau/New_setup/45681_10Vpp_25.avi"
filenames.sort()
print(filenames)
num_top_keys = 3
num_full_keys = 3
num_tran_keys = 2
trans_threshes = np.linspace(20, 60, 4)#[30, 40, 50]#[5, 8, 10, 19, 27, 38, 52, 60]
class_threshes =[15,22,30]
trans_runs = {}
#trans_fil = 10

ave = np.average
expected_blobs = 4
expected_connections = 5
p_bond = 15*1.4
m_bond = 15*1.65#1.6
    
"""area_threshold = 1800   # must have enough particles
convex_thresholds = (2000, 2800)   # convex hull area (> 2500 filtered)  # particles must be in parallelogram
minor_thresholds = (38, 48) # minor_axis length (< filterd) # particles must be in parallelogram
major_thresholds = (59, 76)"""
thresholds = [expected_blobs , expected_connections, p_bond, m_bond]
params = [3, 115, 98, 2, 95, 80]
print("Number files: ", len(filenames))
index = 0
for trans_fil in trans_threshes:
    
    for class_thresh in class_threshes:
        
        Top_bottoms = np.zeros((len(filenames), num_top_keys))
        Full_classif = np.zeros((len(filenames), num_full_keys))
        N_transitions = np.zeros((len(filenames), num_tran_keys, num_tran_keys))
        num_total_frames = 0 
        filtrate_len = 0
        for vidnum in range(len(filenames)):
            print("Processing vid %s : %s" %(vidnum, filenames[vidnum]))
            #meta = ffprobe(filenames[vidnum])
            #print(meta)#['@nb_frames'])
            #print(5)
            frames = vreader(filenames[vidnum])
            #frame_vid = frames[:, :, :, 2]# making videos ino a frame list
            #print(frames)



            sides = {"t":[], "c":[], "ucf":[]}
            classes = {"t":[], "c":[], "ucf":[]}
            transitions = [("t->t",[]), ("t->c",[]),
                          ("c->t" ,[]), ("c->c",[])]

            transitions = OrderedDict(transitions)
            broken_count = 0
            class_num = 0
            origin = 'ucf'
            last_class = 'ucf'
            last_whole = 0
            for fr in frames:
                frame = fr[:, :, 2]
                num_total_frames += 1
                org = total_threshold_filter(frame, num_total_frames, trans_fil, class_thresh,
                                             broken_count, class_num, origin, last_class, last_whole, filtrate_len, 
                                             sides, classes, transitions)
                broken_count, class_num, origin, last_class, last_whole, filtrate_len = org
            filtered_len = filtrate_len
            #----------------------------------------------------------------
            trans_len = len([y for x in transitions.values() for y in x])
            skeys = list(sides.keys())
            ckeys = list(classes.keys())
            tkeys = list(transitions.keys())
            slcs = [len(sides[x]) for x in skeys]
            clcs = [len(classes[x]) for x in ckeys]
            trans = [len(transitions[x]) for x in tkeys] #num of each transition
            ucf = len(sides["ucf"])


            Top_bottoms[vidnum,:] = [slcs[i] for i in range(len(skeys))]
            Full_classif[vidnum,:] = [clcs[i] for i in range(len(ckeys))]
            N_transitions[vidnum,:] = np.reshape(trans, (num_tran_keys, num_tran_keys))
            print("T threshold: %s" %trans_fil)
            print("C threshold: %s" %class_thresh)
            print(N_transitions[vidnum,:])
            #print("finished vid %s" %vidnum)
        trans_runs[index] = [[Top_bottoms, Full_classif, N_transitions, (trans_fil, class_thresh), num_top_keys, num_full_keys,
                                 num_tran_keys, skeys, ckeys, num_tran_keys, num_total_frames], thresholds, params]
        index += 1
        #---------------------------------------------------------------------------------

#-----------------------------------------------------------------------

np.save('trans_runs_plas15.5.npy',trans_runs)


['./tchakamau\\Washed15vplastic\\45659_15Vpp_7.avi', './tchakamau\\Washed15vplastic\\45660_15Vpp_3.avi', './tchakamau\\Washed15vplastic\\45660_15Vpp_4.avi', './tchakamau\\Washed15vplastic\\45660_15Vpp_5_separate.avi', './tchakamau\\Washed15vplastic\\45660_15Vpp_6.avi', './tchakamau\\Washed15vplastic\\45661_15Vpp_11.avi', './tchakamau\\Washed15vplastic\\45661_15Vpp_18.avi', './tchakamau\\Washed15vplastic\\45661_15Vpp_8.avi', './tchakamau\\Washed15vplastic\\45688_15Vpp_12.avi', './tchakamau\\Washed15vplastic\\45697_15Vpp_15.avi', './tchakamau\\Washed15vplastic\\45697_15Vpp_16.avi', './tchakamau\\Washed15vplastic\\45697_15Vpp_17.avi', './tchakamau\\Washed15vplastic\\45731_15Vpp_0.avi', './tchakamau\\Washed15vplastic\\45748_15Vpp_1.avi', './tchakamau\\Washed15vplastic\\45753_15Vpp_10.avi', './tchakamau\\Washed15vplastic\\45753_15Vpp_9.avi', './tchakamau\\Washed15vplastic\\45760_15Vpp_13.avi', './tchakamau\\Washed15vplastic\\45760_15Vpp_14.avi', './tchakamau\\Washed15vplastic\\45779_15Vpp_1

T threshold: 33.333333333333336
C threshold: 15
[[3. 8.]
 [8. 2.]]
Processing vid 3 : ./tchakamau\Washed15vplastic\45660_15Vpp_5_separate.avi
T threshold: 33.333333333333336
C threshold: 15
[[3. 1.]
 [0. 0.]]
Processing vid 4 : ./tchakamau\Washed15vplastic\45660_15Vpp_6.avi
T threshold: 33.333333333333336
C threshold: 15
[[6. 1.]
 [1. 0.]]
Processing vid 5 : ./tchakamau\Washed15vplastic\45661_15Vpp_11.avi
T threshold: 33.333333333333336
C threshold: 15
[[6. 1.]
 [1. 0.]]
Processing vid 6 : ./tchakamau\Washed15vplastic\45661_15Vpp_18.avi
T threshold: 33.333333333333336
C threshold: 15
[[10.  4.]
 [ 3.  2.]]
Processing vid 7 : ./tchakamau\Washed15vplastic\45661_15Vpp_8.avi
T threshold: 33.333333333333336
C threshold: 15
[[1. 1.]
 [0. 0.]]
Processing vid 8 : ./tchakamau\Washed15vplastic\45688_15Vpp_12.avi
T threshold: 33.333333333333336
C threshold: 15
[[1. 0.]
 [0. 0.]]
Processing vid 9 : ./tchakamau\Washed15vplastic\45697_15Vpp_15.avi
T threshold: 33.333333333333336
C threshold: 15
[[3.